<a href="https://colab.research.google.com/github/RussiANC00lHACKA/StartML/blob/main/LDAandGauss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv('baby-names.csv')

In [ ]:
df.head()

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy


In [ ]:
X = df.drop(['sex'], axis=1)
y = df['sex']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=1
)

In [ ]:
from collections import defaultdict
from math import log

def train(samples):
    classes, freq = defaultdict(lambda:0), defaultdict(lambda:0)
    for feats, label in samples:
        classes[label] += 1                 # count classes frequencies
        for feat in feats:
            freq[label, feat] += 1          # count features frequencies

    for label, feat in freq:                # normalize features frequencies
        freq[label, feat] /= classes[label]
    for c in classes:                       # normalize classes frequencies
        classes[c] /= len(samples)
    print(classes, freq)
    return classes, freq                    # return P(C) and P(O|C)

def classify(classifier, feats):
    classes, prob = classifier
    try:
        return min(classes.keys(),              # calculate argmin(-log(C|O)) -> argmax(P(C|O))
            key = lambda cl: -log(classes[cl]) + \
                sum(-log(prob.get((cl,feat))) for feat in feats))
    except:
        return max(classes.keys(),key = lambda x: classes.get(x,0))

def get_features(sample): return (sample[-1]) # get last letter

name = list(X_train['name'])
rez = list(y_train)
features = [(get_features(name[i]), rez[i]) for i in range(len(X_train))]
classifier = train(features)

defaultdict(<function train.<locals>.<lambda> at 0x00000269AC9B1940>, {'boy': 0.49859911406423035, 'girl': 0.5014008859357697}) defaultdict(<function train.<locals>.<lambda> at 0x00000269AD5476A0>, {('boy', 'm'): 0.0164802825191289, ('girl', 'a'): 0.38419489138957297, ('boy', 'y'): 0.11194154163936611, ('boy', 'c'): 0.009239619309915932, ('boy', 'e'): 0.13936055615400847, ('boy', 'b'): 0.004053438759758793, ('boy', 'n'): 0.21362177529512366, ('boy', 's'): 0.07881439692604973, ('girl', 'e'): 0.3024637505107506, ('girl', 's'): 0.022450940333285478, ('girl', 'y'): 0.07846233697392688, ('boy', 'd'): 0.07186247182027164, ('boy', 'k'): 0.022477150821237798, ('girl', 'd'): 0.007962187889964993, ('boy', 'l'): 0.08326762690594912, ('girl', 'n'): 0.08047220964518018, ('girl', 'h'): 0.02474793767186068, ('girl', 'i'): 0.0275087517807251, ('boy', 'a'): 0.01593612224727087, ('boy', 'o'): 0.06324474996390773, ('boy', 'p'): 0.0031872244494541738, ('boy', 't'): 0.0526169666951703, ('girl', 'l'): 0.034

In [ ]:
pred = [classify(classifier, get_features(n)) for n in X_test['name']]


## Средняя доля правильных ответов классификатора и другие метрики оценивания

In [ ]:
np.mean(pred == y_test)

0.7748708010335917

In [ ]:
# Помимо средней доли, мы можем оценить качество модели посчитая Precision(Точность), Полноту(Recall), F1-меру.
# Существуют и другие методы оценивая, например, ROC-кривая + AUC-ROC, Матрица ошибок и т.д.
from sklearn.metrics import precision_score, recall_score, f1_score

print('Precision:', precision_score(y_test, pred, pos_label="girl"))
print('Recall:', recall_score(y_test, pred, pos_label="girl"))
print('F1-мера:', f1_score(y_test, pred, pos_label="girl"))

Precision: 0.8095841187558906
Recall: 0.7149322443883788
F1-мера: 0.7593198801088411


## Модифицирование функции get_features()

In [ ]:
def get_features_2(sample):
    return (sample[:-1]) #Имя целиком без последней буквы
features = [(get_features_2(name[i]), rez[i]) for i in range(len(X_train))]
classifier = train(features)
pred = [classify(classifier, get_features_2(n)) for n in X_test['name']]

defaultdict(<function train.<locals>.<lambda> at 0x00000269B8677560>, {'boy': 0.49859911406423035, 'girl': 0.5014008859357697}) defaultdict(<function train.<locals>.<lambda> at 0x00000269AC9B3880>, {('boy', 'W'): 0.03584794607260653, ('boy', 'i'): 0.34232123224538297, ('boy', 'l'): 0.3048630159805435, ('boy', 'a'): 0.4605150643552811, ('girl', 'A'): 0.09030070787273752, ('girl', 'n'): 0.41121773988713795, ('girl', 'i'): 0.5090941216745994, ('girl', 't'): 0.21851291508840126, ('girl', 'N'): 0.026437555906485704, ('girl', 'a'): 0.44399412498757634, ('girl', 'l'): 0.39964440714277827, ('boy', 'K'): 0.027030328606172334, ('boy', 'e'): 0.47072084578053686, ('boy', 'n'): 0.2548557975279576, ('boy', 'E'): 0.06623207880329161, ('boy', 'r'): 0.42183526380667874, ('boy', 'M'): 0.06756471620376026, ('boy', 'x'): 0.005863604562062035, ('boy', 'B'): 0.06153453196663965, ('boy', 'o'): 0.33128255244483434, ('boy', 'A'): 0.08279009850411452, ('girl', 'M'): 0.10304462579925569, ('girl', 'o'): 0.2129360

In [ ]:
print('Precision:', precision_score(y_test, pred, pos_label="girl"))
print('Recall:', recall_score(y_test, pred, pos_label="girl"))
print('F1-мера:', f1_score(y_test, pred, pos_label="girl"))
print('Cредняя доля:', np.mean(pred == y_test))

Precision: 0.6093356897806425
Recall: 0.76224412828049
F1-мера: 0.6772665295463475
Cредняя доля: 0.6391472868217054


## Модификация метода classify()

In [ ]:
def classify_2(classifier, feats):
    classes, prob = classifier
    try:
        return max(classes.keys(),
            key = lambda cl: classes[cl] + sum(prob.get((cl,feat)) for feat in feats))
    except:
        return min(classes.keys(),key = lambda x: classes.get(x,0))

In [ ]:
features = [(get_features(name[i]), rez[i]) for i in range(len(X_train))]
classifier = train(features)
pred = [classify_2(classifier, get_features(n)) for n in X_test['name']]

defaultdict(<function train.<locals>.<lambda> at 0x00000269B8674A40>, {'boy': 0.49859911406423035, 'girl': 0.5014008859357697}) defaultdict(<function train.<locals>.<lambda> at 0x00000269AC9B1940>, {('boy', 'm'): 0.0164802825191289, ('girl', 'a'): 0.38419489138957297, ('boy', 'y'): 0.11194154163936611, ('boy', 'c'): 0.009239619309915932, ('boy', 'e'): 0.13936055615400847, ('boy', 'b'): 0.004053438759758793, ('boy', 'n'): 0.21362177529512366, ('boy', 's'): 0.07881439692604973, ('girl', 'e'): 0.3024637505107506, ('girl', 's'): 0.022450940333285478, ('girl', 'y'): 0.07846233697392688, ('boy', 'd'): 0.07186247182027164, ('boy', 'k'): 0.022477150821237798, ('girl', 'd'): 0.007962187889964993, ('boy', 'l'): 0.08326762690594912, ('girl', 'n'): 0.08047220964518018, ('girl', 'h'): 0.02474793767186068, ('girl', 'i'): 0.0275087517807251, ('boy', 'a'): 0.01593612224727087, ('boy', 'o'): 0.06324474996390773, ('boy', 'p'): 0.0031872244494541738, ('boy', 't'): 0.0526169666951703, ('girl', 'l'): 0.034

In [ ]:
print('Precision:', precision_score(y_test, pred, pos_label="girl"))
print('Recall:', recall_score(y_test, pred, pos_label="girl"))
print('F1-мера:', f1_score(y_test, pred, pos_label="girl"))
print('Cредняя доля:', np.mean(pred == y_test))

Precision: 0.7948146704168871
Recall: 0.7423466070174526
F1-мера: 0.7676851901662274
Cредняя доля: 0.7768217054263565


## Улучшилась ли доля правильных ответов?

### При модификации classify() средняя точность чуть-чуть выросла, но при изменении get_features() средняя точность резко упала. Изменение классификатора не сильно меняет точность, но если бы мы изменили бы функцию train(), то точность резко бы изменилась. Также насчет изменения точности при изменении get_features() можно сказать, что выбор целевого признака в виде всего слова кроме последней буквы очень плохо сказался на точности, я это связываю просто с неудачным выбором целевого признака.

## Запуск гауссовского и мультиномиального классификатор

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import LabelEncoder
df['sex'] = df['sex'].astype('category').cat.codes
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])
X = df.drop(['sex'], axis=1)
y = df['sex']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=1
)
gnb = GaussianNB()
pred = gnb.fit(X_train, y_train).predict(X_test)
mnb = MultinomialNB()
pred_2 = mnb.fit(X_train,y_train).predict(X_test)
print(pred,pred_2)

[1 1 0 ... 0 1 1] [1 1 0 ... 0 1 1]


### Гауссовский классификатор

In [ ]:
print('Cредняя доля:', np.mean(pred == y_test))
print('Precision:', precision_score(y_test, pred, pos_label=1))
print('Recall:', recall_score(y_test, pred, pos_label=1))
print('F1-мера:', f1_score(y_test, pred, pos_label=1))

Cредняя доля: 0.5070155038759689
Precision: 0.5026891691394659
Recall: 0.7049964886727182
F1-мера: 0.5868979180876287


### Мультиномиальный классификатор

In [ ]:
print('Cредняя доля:', np.mean(pred_2 == y_test))
print('Precision:', precision_score(y_test, pred_2, pos_label=1))
print('Recall:', recall_score(y_test, pred_2, pos_label=1))
print('F1-мера:', f1_score(y_test, pred_2, pos_label=1))

Cредняя доля: 0.5233462532299742
Precision: 0.5203307342090114
Recall: 0.5172315135121076
F1-мера: 0.5187764951411987


## Лучше всех как видно по доле правильных ответов показал себя наивный классификатор. Все остальные из sklearn дали результат хуже


# Задание 2

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df.to_csv('iris_dataset.csv', index=False)

In [ ]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [ ]:
X = iris_df.drop(['target'], axis=1)
y = iris_df['target']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=1
)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
pred = lda.fit(X_train, y_train).predict(X_test)
print(pred)


[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]


In [ ]:
print('Cредняя доля правильных ответов:', np.mean(pred == y_test))
print('Precision(Точность):', precision_score(y_test, pred,average = 'weighted'))
print('Recall(Полнота):', recall_score(y_test, pred,average = 'weighted'))
print('F1-мера:', f1_score(y_test, pred ,average = 'weighted'))

Cредняя доля правильных ответов: 1.0
Precision(Точность): 1.0
Recall(Полнота): 1.0
F1-мера: 1.0


### Здесь выдается идеальный результат, что является следствием того, что load_iris достаточно простой датасет, который спокойно линейно разделим. Так что можно посмотреть также эффективность по различным оценкам на кросс-валидации.

In [ ]:
from sklearn.model_selection import cross_val_score
print('Средняя доля правильных ответов:',np.mean(cross_val_score(lda, X, y, cv= 5, scoring="accuracy")))
print('Precision(Точность):',np.mean(cross_val_score(lda, X, y, cv= 5, scoring="precision_macro")))
print('Recall(Полнота):',np.mean(cross_val_score(lda, X, y, cv= 5, scoring="recall_macro")))
print('F1-мера:',np.mean(cross_val_score(lda, X, y, cv= 5, scoring="f1_macro")))

Средняя доля правильных ответов: 0.9800000000000001


Precision(Точность): 0.9806060606060605
Recall(Полнота): 0.9800000000000001
F1-мера: 0.9799832915622388


### Реализация из файла Sem3.ipynb

In [ ]:
def LDA_dimensionality(X, y, k):
    '''
    X - набор данных, y - метка, k - целевой размер
    '''
    label_ = list(set(y))

    X_classify = {}

    for label in label_:
        X1 = np.array([X[i] for i in range(len(X)) if y[i] == label])
        X_classify[label] = X1

    mju = np.mean(X, axis=0)
    mju_classify = {}

    for label in label_:
        mju1 = np.mean(X_classify[label], axis=0)
        mju_classify[label] = mju1

    #St = np.dot((X - mju).T, X - mju)

    Sw = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sw += np.dot((X_classify[i] - mju_classify[i]).T,
                     X_classify[i] - mju_classify[i])

    # Sb=St-Sw

    Sb = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sb += len(X_classify[i]) * np.dot((mju_classify[i] - mju).reshape(
            (len(mju), 1)), (mju_classify[i] - mju).reshape((1, len(mju))))

    eig_vals, eig_vecs = np.linalg.eig(
        np.linalg.inv(Sw).dot(Sb))  # Вычислить собственное значение и собственную матрицу Sw-1 * Sb

    sorted_indices = np.argsort(eig_vals)
    topk_eig_vecs = eig_vecs[:, sorted_indices[:-k - 1:-1]]  # Извлекаем первые k векторов признаков
    return topk_eig_vecs


### Данная функция реализует метод Линейного Дискриминантного Анализа (LDA) для снижения размерности данных и проекции их на пространство меньшей размерности. Однако, для оценки точности классификации на датасете Iris после уменьшения размерности, нам потребуется дополнительно вычислить точность. Которую мы будем вычислять и другие оценки используя кросс-валидацию.

In [ ]:
X = iris.data
y = iris.target
lda_2 = LDA_dimensionality(X, y, 2)

### Заметим, что MultinomialNB принимает на вход только неотрицательные значения. Если используется LinearDiscriminantAnalysis (LDA) для преобразования данных, результаты могут быть отрицательными значениями, что может вызвать проблему при использовании MultinomialNB. Поэтому будем использовать GaussianNB()

In [ ]:
mnb_2 = GaussianNB() # саму точность и другие параметры будем вычислять через знакомую из 1 части модель
X = X.dot(lda_2)
print('Средняя доля правильных ответов:',np.mean(cross_val_score(mnb_2, X, y, cv= 5, scoring="accuracy")))
print('Precision(Точность):',np.mean(cross_val_score(mnb_2, X, y, cv= 5, scoring="precision_macro")))
print('Recall(Полнота):',np.mean(cross_val_score(mnb_2, X, y, cv= 5, scoring="recall_macro")))
print('F1-мера:',np.mean(cross_val_score(mnb_2, X, y, cv= 5, scoring="f1_macro")))

Средняя доля правильных ответов: 0.9800000000000001
Precision(Точность): 0.9806060606060605
Recall(Полнота): 0.9800000000000001
F1-мера: 0.9799832915622388


### Результаты нашего LDA и LDA из sklearn дают одинаковые высокие значения на кросс-валидации по всем параметрам. Для того, чтобы еще попытаться улучшить данный результат постраемся поменять параметр solver и другие

### Рассмотрим основные параметры и их влияние:

### 1) Solver (solver): Этот параметр определяет метод решения для обучения LDA. Например, svd использует сингулярное разложение, lsqr и eigen используют итеративные методы. svd может быть более подходящим для больших наборов данных, lsqr и eigen могут быть эффективны для данных с меньшим числом объектов.

### 2) Shrinkage (shrinkage): Регуляризация позволяет улучшить обобщение модели и уменьшить переобучение. Относительно solver, он может улучшить производительность, особенно когда данных ограниченное количество.

### Найдем с помощью GridSearchCV лучшие значения параметров

In [ ]:
import warnings
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
iris = load_iris()
lda = LinearDiscriminantAnalysis()
parameters = {'solver': ['svd', 'lsqr', 'eigen'], 'shrinkage': [None, 'auto']}
clf = GridSearchCV(lda, parameters, cv=5, scoring='accuracy')
clf.fit(iris.data, iris.target)
print("Наилучшие параметры:", clf.best_params_)

Наилучшие параметры: {'shrinkage': None, 'solver': 'svd'}


### По итогу с помощью GridSearchCV стало ястно, что параметры: 'shrinkage': None, 'solver': 'svd' в LDA() являются наилучшими для данного датасета.

### Видимо, solver выбирается svd, потому что размер данных уже не мал, а shrinkage оставляет None, потому что никакого прироста производительности не появляется